#### Example how to read the data in MNE-python 


In [1]:
# import mne
# import numpy as np
# import matplotlib as plt




# import matplotlib.pyplot as plt
# from mne.time_frequency import tfr_multitaper

# from utils_dataset2mne import data2mne



In [2]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import asrpy

from utils_dataset2mne import data2mne
from utils_dataset2mne import create_name
from utils_dataset2mne import apply_ASR
from utils_dataset2mne import plot_TF_M
from utils_dataset2mne import plot_TF_M_stat
from utils_dataset2mne import plot_ERDs

mne.set_log_level(verbose='WARNING')

%matplotlib qt

#### Creating Epochs

In [3]:
subject = 41
fname_mat = create_name(subject,'EEG_Data')

# Convert .mat data to a raw object of MNE-python
raw = data2mne(fname_mat, scale=1e-8)

# Event ID definition
event_id = {'Blink':1,'Up/Down':2,'Left/Right':3,'Jaw':4,'HeadMov':5, 'Resting':6, 'ME_Right':7, 'ME_Left':8, 'MI_Right':9, 'MI_Left':10}

# Read events from Stimuli channel
events = mne.find_events(raw, stim_channel='Stim', verbose=True,initial_event=True)

# Plot Raw Data
# raw.plot(events=events,event_id=event_id)

# Only imaginary events
event_id_MI = {'MI_Right':9, 'MI_Left':10}
events_MI = events[events[:,2] > 8]

# Change the custom montage to standard positions for better visualization
biosemi64_montage = mne.channels.make_standard_montage('biosemi64')
raw.set_montage(biosemi64_montage)

# Create Epochs
epochs = mne.Epochs(raw, events_MI, event_id_MI, tmin=-1.95, tmax=4.95, picks=('eeg'), baseline=None, preload=True)
epochs.filter(0.5,None)


# Common Average Reference as a projection
epochs.set_eeg_reference(ref_channels='average',projection=True)

# Plot Raw Data only with imaginary events
# raw.plot(events=events_MI,event_id=event_id_M)

# Electrode position 2D
# raw.plot_sensors(kind='topomap',ch_type='eeg', show_names=True, sphere='eeglab')


246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]


Number of events,200
Events,MI_Left: 100MI_Right: 100
Time range,-1.949 – 4.949 sec
Baseline,off


In [4]:
# # Plot Raw Data
# raw.plot(events=events,event_id=event_id)

#### Applying ASR - Step by Step

In [5]:
# # Creating ASR object
# asr = asrpy.ASR(sfreq=raw.info["sfreq"])

# # Filter the data
# filtered = raw.load_data().copy().filter(0.5,None)

# # Fit the ASR method with RS dataca
# asr.fit(filtered.copy().crop(tmin=51,tmax=115))

# # Applying the method to filtered data
# cleaned = asr.transform(filtered)

# # Save clean data
# fname_ASR = create_name(subject,'Derivates',type='ASR')
# cleaned.save(fname_ASR)



#### Applying ASR - function

In [6]:
# # Applying ASR to the raw data (no filtering before)
# # tmin and tmax define the period to use for calibration
# # the output is a raw object of MNE-python with the cleaned data
# cleaned = apply_ASR(raw,tmin=51,tmax=115)
# fname_ASR = create_name(subject,'Derivates',type='ASR')
# cleaned.save(fname_ASR)

#### Create Epochs Cleaned by ASR

In [7]:
# Read previously cleaned data by ASR
fname_ASR = create_name(subject,'Derivates',type='ASR')
cleaned = mne.io.read_raw_fif(fname_ASR)

# Create cleaned Epochs
epochs_ASR = mne.Epochs(cleaned, events_MI, event_id_MI, tmin=-1.95, tmax=4.95, picks=('eeg'), baseline=None, preload=True)

# Common Average Reference as a projection
epochs_ASR.set_eeg_reference(ref_channels='average',projection=True)

C:\Users\andresfs\AppData\Local\Temp\ipykernel_14284\1107146597.py:3: RuntimeWarning: This filename (Derivates\s41_ASR.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  cleaned = mne.io.read_raw_fif(fname_ASR)


Number of events,200
Events,MI_Left: 100MI_Right: 100
Time range,-1.949 – 4.949 sec
Baseline,off


In [8]:
# # Plot epochs as an image an compare 
# epochs_ASR.plot_image(picks="eeg", combine="mean",title='Cleaned ASR')
# epochs.plot_image(picks="eeg", combine="mean")

#### Time-Frequency Analysis

In [9]:
# Prepare Epochs for TF
channels = ['C3','C4']
epochs_TF_raw = epochs.copy().apply_proj().pick_channels(channels)
epochs_TF_ASR = epochs_ASR.copy().apply_proj().pick_channels(channels)


In [10]:
# Compute TF Maps - Wavelet
freqs = np.arange(2, 35.5, 0.5)
n_cycles = freqs / 2.0  # different number of cycle per frequency
TF_raw_wav = mne.time_frequency.tfr_morlet(epochs_TF_raw['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)
TF_ASR_wav = mne.time_frequency.tfr_morlet(epochs_TF_ASR['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)

In [11]:
# Compute TF Maps - Multitaper
TF_raw_mtaper = mne.time_frequency.tfr_multitaper(epochs_TF_raw['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)
TF_ASR_mtaper = mne.time_frequency.tfr_multitaper(epochs_TF_ASR['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)

In [12]:
# Plot TF Maps - Wavelet
event_ids = ['MI_Right','MI_Left'] # events 
kwargs = dict(baseline=(-1,0), mode='percent', vmin=-1.2, vmax=1.2, fmin=None, fmax=None, cmap='jet')
plot_TF_M(TF_raw_wav, event_ids, **kwargs, comment='raw - Wavelet - s' + str(subject))
#plot_TF_M(TF_ASR_wav, event_ids, **kwargs, comment='ASR - Wavelet - s' + str(subject))


In [13]:
# Plot TF Maps - Multitaper
plot_TF_M(TF_raw_mtaper, event_ids, **kwargs, comment='raw - Multitaper - s' + str(subject))
#plot_TF_M(TF_ASR_mtaper, event_ids, **kwargs, comment='ASR - Multitaper - s' + str(subject))


In [14]:
channels = ['C3','C4']
plot_ERDs(TF_raw_mtaper,channels, comment='raw - Multitapers - s' + str(subject))

In [15]:
# Plot ERD maps - Multitaper - Statistically significant
kwargs = dict(baseline=(-1,0), mode='percent', vmin=-1.2, vmax=1.2, cmap='jet')
plot_TF_M_stat(TF_raw_mtaper.copy(), event_ids, **kwargs, comment='raw - Multitaper - s' + str(subject))
#plot_TF_M_stat(TF_ASR_mtaper.copy(), event_ids, **kwargs, comment='ASR - Multitaper - s' + str(subject))

#### Applying ICA to epochs

In [16]:
#Defining ICA
# ica = mne.preprocessing.ICA(n_components=40, max_iter="auto", random_state=97)

#Fit ICA to the epochs
# ica.fit(epochs)

In [17]:
# Plot a topography map of the components
# ica.plot_components()

In [18]:
# Plot the time courses of the source separation (sources in ICA does not mean brain sources)
# ica.plot_sources(epochs, show_scrollbars=False)

In [19]:
# Plot the properties of particular components
### components = [i for i in range(0, 40)]
# ica.plot_properties(epochs,[0,1,2,7])

In [20]:
# Exclude components 0 and 6
# ica.exclude = [0,1,2] # This is decided manuallyt after analyzing the components

In [21]:
#Apply ICA to the data
# ica.apply(epochs)